In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing
data = pd.read_csv('/Users/voronik/DeepLearning/Anti_duplicate/cleaned.csv')
data = data.dropna()

In [42]:
y = data['is_duplicate'].values
X = data.drop('is_duplicate', axis=1)

In [5]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 4.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 3.3 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [25]:
data = (X['name_1'] + X['name_2']).unique()

In [30]:
data

array(['iko industries ltdenormous industrial trade pvt ltd',
       'apcotex industries ltdtechnocraft industries india ltd',
       'rishichem distributors pvt ltddsa', ...,
       'xeikon industrial co ltd dongguan cityyi cheng trading co ltd dongguan city',
       'shanghai kechuan trading co ltdshanghai mg stationery inc',
       'dih wei industries coburlington industries corporate'],
      dtype=object)

In [31]:
from gensim.models.phrases import Phrases, Phraser

In [32]:
sent = [row.split() for row in data]

In [36]:
phrases = Phrases(sent, min_count=30, progress_per=100)

In [47]:
sentences = phrases[sent]

In [49]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

195476

In [50]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['co',
 'ltd',
 'de',
 'international',
 'industries',
 'sa',
 'inc',
 'trading',
 'logistics',
 'pvt']

In [51]:
import multiprocessing

from gensim.models import Word2Vec

In [54]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [56]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [58]:
t = time()

w2v_model.build_vocab(sentences, progress_per=100)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.17 mins


In [59]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 5.52 mins


In [60]:
w2v_model.save('w2v.pkl')

In [61]:
w2v_model.init_sims(replace=True)

/var/folders/1t/dkl6p5jx5k5cggckx5qn1zx80000gn/T/ipykernel_38169/514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)
